In [ ]:
from utils.events_utils import get_manipulation_EVs, get_ev_vars, normalize_rt
import pandas as pd
import os 
from os import path
from glob import glob 
import csv
import numpy as np
import json

In [ ]:
data_dir = '/data/event_files'
ppi_dir = '/data/ppi_1'
os.mkdir(ppi_dir)

In [ ]:
be_subs = ['65','91','225','283','300','413','579','595','630','685','725','889','1025',
        '1046','1132','1211','1237','1352','1961','1976','2199','2604','2689','2800',
        '2834','2839','3002','3010','3225','3324','3364','3464','3525','4022','4081',
        '4129','4591','4592','4829','4930','5064','5293','5387','5469','5497','5579',
        '5906','6018','6638','7124']

In [ ]:
#files = glob(os.path.join(data_dir, "*manipulation*")) ##finds ALL reds , not just BE folks 

In [ ]:
files = []
for sub in be_subs:
    files.extend(glob(path.join(data_dir, "{}_manipulation*".format(sub))))

In [ ]:
def get_ppi_EVs(events_df, regress_rt=True): 
    output_dict = {
            'conditions': [],
            'onsets': [],
            'durations': [],
            'amplitudes': []
            }
    mean_rt = calc_mean_rt(files)
    get_ev_vars(output_dict, events_df, 
               condition_spec = [('cue', 'np_cue')],
               col = 'trial_id',
               duration = 'duration',
               subset='trial_type!="no_stim" and junk==False') 

    # cue regressor 
    get_ev_vars(output_dict, events_df, 
               condition_spec = [('cue_future', 'cue_future')],
               col = 'trial_type',
               duration = 'duration',
               amplitude = 1,
               subset='trial_type!="no_stim" and junk==False')
    
    get_ev_vars(output_dict, events_df, 
               condition_spec = [('cue_present', 'cue_present')],
               col = 'trial_type',
               duration = 'duration',
               amplitude = 1,
               subset='trial_type!="no_stim" and junk==False')
    #demean probe regressor 

    get_ev_vars(output_dict, events_df, 
              condition_spec = [('probe', 'np_probe')],
               col = 'trial_id',
               duration = 'duration',
               subset='trial_type!="no_stim" and junk==False')

    get_ev_vars(output_dict, events_df, 
                condition_spec = [('valence_probe', 'valence_probe')],
                col = 'trial_type',
               duration = 'duration',
               amplitude = 1, 
               subset='trial_type!="no_stim" and junk==False')
    
    get_ev_vars(output_dict, events_df, 
                condition_spec = [('neutral_probe', 'neutral_probe')],
                col = 'trial_type',
               duration = 'duration',
               amplitude = 1, 
               subset='trial_type!="no_stim" and junk==False')

    #calculate the mean response for this participant
    response_mean = np.nanmean(events_df['response'])
    #demean and replace 
    events_df.response = events_df.response - response_mean 
    #rename column 
    events_df = events_df.replace({'current_rating': 'rating'})
    get_ev_vars(output_dict, events_df, 
              condition_spec = [('rating', 'rating')], 
               col = 'trial_id',
               duration = 'response_time',
               amplitude = 'response', 
               subset='junk==False and trial_type!="no_stim" and trial_id=="rating"')

    # nuisance regressors
    get_ev_vars(output_dict, events_df, 
                condition_spec=[(True, 'junk')], 
                col='junk', 
                duration='duration')

    if regress_rt == True:
        #events_df["mean_rt"] = np.mean(events_df.response_time)
        normalize_rt(events_df)
        get_ev_vars(output_dict, events_df, 
                condition_spec='response_time', 
                duration= float(mean_rt), 
                amplitude='response_time', #demeaned rt after running normalize_rt
                subset='junk==False and trial_type!="no_stim" and trial_id=="rating"') #the only trials that will have responses 

    return output_dict

def calc_mean_rt(files):
    """for calculating mean rts across all subs, only tested on manipulation"""
    rt_df = pd.DataFrame()
    for file in files:
        temp_df = pd.read_csv(path.join(file), sep = "\t") 
        rt_df = pd.concat([rt_df, temp_df], axis = 0)
    mean_rt = np.nanmean(rt_df['response_time'])
    return mean_rt

In [79]:
for file in files:  #iterate over each path
    event_df = pd.read_csv(path.join(file), sep = "\t") 
    ev = get_ppi_EVs(event_df) #create ev dict
    sub = file.split("/")[-1].split("_")[0] #get subject id from path
    temp_df = pd.DataFrame()
    temp_df = pd.DataFrame.from_dict(ev).set_index('conditions') #create dataframe, set the index 
    ev_df = pd.DataFrame()
    for col in temp_df.columns:
        ev_df[col] = temp_df[col].explode() #make lists in cells "explode"
    for cond in ev['conditions']:
        cond_label = []
        cond_df = pd.DataFrame()
        cond_df = ev_df[ev_df.index == cond]
        print(len(cond_df.index), len(ev_df.index))
        cond_df.to_csv(path.join(ppi_dir, 'sub-{}_{}.txt'.format(sub, cond)), sep = '\t', index=False, header=None) #write to csv

/usr/local/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:85: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



53 53
53 375
25 25
25 375
28 28
28 375
53 53
53 375
28 28
28 375
25 25
25 375
53 53
53 375
57 57
57 375
53 53
53 375
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
72 72
72 432
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
72 72
72 432
71 71
71 429
36 36
36 429
35 35
35 429
71 71
71 429
36 36
36 429
35 35
35 429
71 71
71 429
3 3
3 429
71 71
71 429
70 70
70 426
34 34
34 426
36 36
36 426
70 70
70 426
35 35
35 426
35 35
35 426
70 70
70 426
6 6
6 426
70 70
70 426
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
72 72
72 432
64 64
64 408
29 29
29 408
35 35
35 408
64 64
64 408
31 31
31 408
33 33
33 408
64 64
64 408
24 24
24 408
64 64
64 408
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
36 36
36 432
36 36
36 432
72 72
72 432
72 72
72 432
69 69
69 423
34 34
34 423
35 35
35 423
69 69
69 423
34 34
34 423
35 35
35 423
69 69
69 423
9 9
9 423
69 69
69 423
72 72
72